# Import Libraries 

In [1]:
import os
import re
import torch

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
tqdm.pandas()

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown

import torch
import transformers
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    AutoConfig,
    set_seed
)

# Configuration

In [2]:
!pip install -U bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

In [4]:
set_seed(42)

2024-06-15 21:34:44.935782: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 21:34:44.935892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 21:34:45.052501: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Data

In [82]:
df1 = pd.read_csv("data/combined_dataset.csv")

In [95]:
df2 = pd.read_csv("data/AIMO/data.csv")

In [83]:
def clean_output(txt):
    try:
        txt = txt[-30:]
        pattern = r"(\d+)$"
        ans_cln = re.sub(r"\D", " ", txt).strip()
        matches = re.findall(pattern, ans_cln)
        return int(matches[0])
    except:
        return np.NaN

In [84]:
ans = []
for i, sol in df1[df1['answers'].isna()][['solutions']].iterrows():
    df1.loc[i, 'answers'] = clean_output(sol['solutions'])

In [85]:
df1 = df1.rename(columns={"questions":"problem", "answers":"answer", "solutions":"solution"})

In [174]:
def clean_sol(text):
    return re.sub(r'[\-~][^\s]*$', '', text).strip()


In [178]:
df1['solution'] = df1['solution'].fillna("").apply(clean_sol).values

In [179]:
df1 = df1[~df1['problem'].isna()].copy()

In [180]:
df1.shape

(13502, 3)

In [181]:
df = df1.copy()

In [182]:
def is_integer(text):
    try:
        if int(text) >= 0:
            return True
        else:
            return False
    except ValueError:
        return False
    
df["is_integer"] = df.answer.map(is_integer)
df = df[df.is_integer].reset_index(drop=True)
df.head(2)

,problem,solution,answer,is_integer
0,Every morning Aya goes for a $9$ -kilometer-lo...,$\frac{9}{s} + t = 4$ in hours and $\frac{9}{s...,204,True
1,Alice and Bob play the following game. A stack...,Let's first try some experimentation. Alice ob...,809,True


In [88]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative."""

In [186]:
df.shape

(8300, 4)

In [185]:
df['solution'].values

array(['$\\frac{9}{s} + t = 4$ in hours and $\\frac{9}{s+2} + t = 2.4$ in hours. Subtracting the second equation from the first, we get, $\\frac{9}{s} - \\frac{9}{s+2} = 1.6$  Multiplying by $(s)(s+2)$ , we get $9s+18-9s=18=1.6s^{2} + 3.2s$  Multiplying by 5/2 on both sides, we get $0 = 4s^{2} + 8s - 45$  Factoring gives us $(2s-5)(2s+9) = 0$ , of which the solution we want is $s=2.5$ . Substituting this back to the first equation, we can find that $t = 0.4$ hours. Lastly, $s + \\frac{1}{2} = 3$ kilometers per hour, so $\\frac{9}{3} + 0.4 = 3.4$ hours, or $\\framebox{204}$ minutes',
       "Let's first try some experimentation. Alice obviously wins if there is one coin. She will just take it and win. If there are 2 remaining, then Alice will take one and then Bob will take one, so Bob wins. If there are $3$ , Alice will take $1$ , Bob will take one, and Alice will take the final one. If there are $4$ , Alice will just remove all $4$ at once. If there are $5$ , no matter what Alice does

In [89]:
df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.problem,
                                                             solution=f"{row.solution}\n\nAnswer:\n{row.answer}"),
                                                             axis=1)
data = df.prompt.tolist()

  0%|          | 0/8301 [00:00<?, ?it/s]

In [90]:
MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True


tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors.index.json:   0%|          | 0.00/23.8k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [91]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map="auto",
)

In [92]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [93]:
def extract_output(ans):
    pattern = r"(\d+)$"
    ans_cln = re.sub(r"\D", " ", ans).strip()
    matches = re.findall(pattern, ans_cln)
    return int(matches[0])

In [96]:
df2.iloc[0]['problem']

'Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?'

## Zero Shot

In [151]:
row = df2.iloc[0]

def solve(problem, template):
    prompt = template + f"""\n\nProblem:\n{problem}\n\nSolution:\nAnswer:"""

    messages = [
        {"role": "user", "content": prompt}
    ]

    query_prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False
        )

    raw_output = pipeline(
                    query_prompt, 
                    max_new_tokens=10000, 
                    do_sample=True, 
                    temperature=0.8,
                    return_full_text=False
                )
    raw_output = raw_output[0]['generated_text']
    torch.cuda.empty_cache()
    return extract_output(raw_output)

In [152]:
actual_ans = []
pred_ans = []

for i, row in tqdm(df2[5:10].iterrows()):
    actual_ans.append(row.answer)
    pred_ans.append(solve(row.problem, template))

0it [00:00, ?it/s]

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [153]:
actual_ans, pred_ans

([199, 185, 320, 480, 199], [99, 1967, 400, 219, 375])

accuracy = 0

## Few Shot

In [132]:
template_fs = """Role:
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.

Instruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.

Example 1:
Problem:
Every morning Aya goes for a $9$-kilometer-long walk and stops at a coffee shop afterwards. When she walks at a constant speed of $s$ kilometers per hour, the walk takes her 4 hours, including $t$ minutes spent in the coffee shop. When she walks $s+2$ kilometers per hour, the walk takes her 2 hours and 24 minutes, including $t$ minutes spent in the coffee shop. Suppose Aya walks at $s+\frac{1}{2}$ kilometers per hour. Find the number of minutes the walk takes her, including the $t$ minutes spent in the coffee shop.

Solution:
We start by setting up the equations based on the given conditions. For the first scenario, we have:
$$ \frac{9}{s} + t = 4 \text{ (in hours)} $$
For the second scenario:
$$ \frac{9}{s+2} + t = 2.4 \text{ (in hours)} $$
Subtracting the second equation from the first:
$$ \frac{9}{s} - \frac{9}{s+2} = 1.6 $$
Multiplying both sides by $(s)(s+2)$, we get:
$$ 9s + 18 - 9s = 1.6s^2 + 3.2s $$
Simplifying, we get:
$$ 18 = 1.6s^2 + 3.2s $$
Multiplying by $\frac{5}{2}$ on both sides, we obtain:
$$ 0 = 4s^2 + 8s - 45 $$
Factoring the quadratic equation, we get:
$$ (2s - 5)(2s + 9) = 0 $$
The valid solution is:
$$ s = 2.5 $$
Substituting this back into the first equation:
$$ \frac{9}{2.5} + t = 4 $$
$$ 3.6 + t = 4 $$
$$ t = 0.4 \text{ hours} $$
Now, considering the speed $s + \frac{1}{2} = 3$ kilometers per hour, we calculate the time:
$$ \frac{9}{3} + 0.4 = 3.4 \text{ hours} $$
Converting to minutes:
$$ 3.4 \times 60 = 204 \text{ minutes} $$

Answer:
204

Example 2:
Problem:
Consider the paths of length $16$ that follow the lines from the lower left corner to the upper right corner on an $8\times 8$ grid. Find the number of such paths that change direction exactly four times, as in the examples shown below.

Solution:
We divide the path into eight “$R$” movements and eight “$U$” movements. Five sections of alternative $RURUR$ or $URURU$ are necessary in order to make four “turns.” We use the first case and multiply by $2$. For $U$, we have seven ordered pairs of positive integers $(a,b)$ such that $a+b=8$. For $R$, we subtract $1$ from each section (to make the minimum stars of each section $0$) and we use Stars and Bars to get:
$$ {7 \choose 5} = 21 $$
Thus our answer is:
$$ 7 \times 21 \times 2 = 294 $$

Answer:
294

Example 3:
Problem:
Alice and Bob play the following game. A stack of $n$ tokens lies before them. The players take turns with Alice going first. On each turn, the player removes either $1$ token or $4$ tokens from the stack. Whoever removes the last token wins. Find the number of positive integers $n$ less than or equal to $2024$ for which there exists a strategy for Bob that guarantees that Bob will win the game regardless of Alice's play.

Solution:
Let's first try some experimentation. Alice obviously wins if there is one coin. She will just take it and win. If there are 2 remaining, then Alice will take one and then Bob will take one, so Bob wins. If there are $3$, Alice will take $1$, Bob will take one, and Alice will take the final one. If there are $4$, Alice will just remove all $4$ at once. If there are $5$, no matter what Alice does, Bob can take the final coins in one try. Notice that Alice wins if there are $1$, $3$, or $4$ coins left. Bob wins if there are $2$ or $5$ coins left. After some thought, you may realize that there is a strategy for Bob. If $n$ is a multiple of $5$, then Bob will win. The reason for this is the following: Let's say there are a multiple of $5$ coins remaining in the stack. If Alice takes $1$, Bob will take $4$, and there will still be a multiple of $5$. If Alice takes $4$, Bob will take $1$, and there will still be a multiple of $5$. This process will continue until there are $0$ coins left. For example, let's say there are $205$ coins. No matter what Alice does, Bob can simply just do the complement. After each of them makes a turn, there will always be a multiple of $5$ left. This will continue until there are $5$ coins left, and Bob will end up winning. After some more experimentation, you'll realize that any number that is congruent to $2$ mod $5$ will also work. This is because Bob can do the same strategy, and when there are $2$ coins left, Alice is forced to take $1$ and Bob takes the final coin. For example, let's say there are $72$ coins. If Alice takes $1$, Bob will take $4$. If Alice takes $4$, Bob will take $1$. So after they each make a turn, the number will always be equal to $2$ mod $5$. Eventually, there will be only $2$ coins remaining, and we've established that Alice will simply take $1$ and Bob will take the final coin. So we have to find the number of numbers less than or equal to $2024$ that are either congruent to $0$ mod $5$ or $2$ mod $5$. There are $405$ numbers in the first category: $5, 10, 15, \dots, 2020$. For the second category, there are $404$ numbers: $2, 7, 12, 17, \dots, 2022$. So the answer is:
$$ 405 + 404 = 809 $$

Answer:
809
"""

In [133]:
torch.cuda.empty_cache()

In [134]:
actual_ans = []
pred_ans = []

for i, row in tqdm(df2[5:10].iterrows()):
    actual_ans.append(row.answer)
    pred_ans.append(solve(row.problem, template_fs))

0it [00:00, ?it/s]

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [135]:
actual_ans, pred_ans

([199, 185, 320, 480, 199], [1, 97, 256, 1103, 103])

accuracy = 0

## RAG

In [187]:
from transformers import AutoTokenizer, AutoModel

In [188]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [189]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = AutoModel.from_pretrained('bert-base-uncased').to(device)

In [190]:
df

,problem,solution,answer,is_integer
0,Every morning Aya goes for a $9$ -kilometer-lo...,$\frac{9}{s} + t = 4$ in hours and $\frac{9}{s...,204,True
1,Alice and Bob play the following game. A stack...,Let's first try some experimentation. Alice ob...,809,True
2,Jen enters a lottery by picking $4$ distinct n...,This is a conditional probability problem. Bay...,116,True
3,Rectangles $ABCD$ and $EFGH$ are drawn such th...,We use simple geometry to solve this problem. ...,104,True
4,Consider the paths of length $16$ that follow ...,We divide the path into eight “ $R$ ” movement...,294,True
...,...,...,...,...
8295,Find the number of real roots of\n\[2x^{2001} ...,We can factor the given equation as\n\[(2x + 3...,1,True
8296,"For a complex number $z,$ compute the minimum ...","Geometrically, $|z + 5 - 3i|$ is the distance ...",13,True
8297,Compute the smallest positive integer $x$ grea...,Let $q$ and $r$ be the remainder when $x$ is d...,1700,True
8298,"For positive real numbers $a,$ $b,$ $c,$ and $...","Let $S$ denote the given sum. First, we apply...",9,True


In [142]:
!pip install faiss-gpu

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [143]:
import faiss

In [191]:
encoded_questions = []
for i, item in tqdm(df.iterrows()):
    inputs = bert_tokenizer(item['problem'], return_tensors='pt', max_length=512).to(device)
    outputs = bert_model(**inputs)
    encoded_questions.append(outputs.last_hidden_state.mean(dim=1).cpu().detach().numpy())

encoded_questions = np.vstack(encoded_questions)

index = faiss.IndexFlatL2(768)
index.add(encoded_questions)

0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [192]:
def retrieve_relevant_questions(query, index, dataset, tokenizer, model, top_k=3):
    inputs = tokenizer(query, return_tensors='pt', max_length=512, truncation=True).to(device)
    outputs = model(**inputs)
    query_vector = outputs.last_hidden_state.mean(dim=1).cpu().detach().numpy()

    D, I = index.search(query_vector, k=top_k)
    retrieved_docs = [dataset.iloc[i] for i in I[0]]
    return retrieved_docs

In [195]:
def generate_answer(query, index, dataset, tokenizer, model, template):
    top_k = 3
    retrieved_docs = retrieve_relevant_questions(query, index, dataset, tokenizer, model, top_k)
    
    prompt = template
    for i in range (1, top_k+1):
        prompt += f"\nExample {i}:"
        prompt += f"\nProblem:\n{retrieved_docs[i-1]['problem']}\n"
        prompt += f"\nSolution:\n{retrieved_docs[i-1]['solution']}\n"
        prompt += f"\Answer:\n{retrieved_docs[i-1]['answer']}\n"

    prompt += f"\nNow for you:\nProblem: \n{query}\nSolution:\nAnswer:\n"
    
    messages = [
        {"role": "user", "content": prompt}
    ]

    query_prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False
        )

    raw_output = pipeline(
                    query_prompt, 
                    max_new_tokens=10000, 
                    do_sample=True, 
                    temperature=0.8,
                    return_full_text=False
                )
    raw_output = raw_output[0]['generated_text']
    torch.cuda.empty_cache()
    return extract_output(raw_output)

In [196]:
actual_ans = []
pred_ans = []

for i, row in tqdm(df2[5:10].iterrows()):
    actual_ans.append(row.answer)
    pred_ans.append(generate_answer(row.problem, index, df_cln, bert_tokenizer, bert_model, template))

0it [00:00, ?it/s]

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [197]:
actual_ans, pred_ans

([199, 185, 320, 480, 199], [1, 19, 320, 169, 1])

accuracy = 10%

## Fine Tuninh (PEFT)

to be done